# Downloading all data

This notebook downloads and prepares all final data to testing and exploring processes.

## Importing modules

In [1]:
import pandas as pd
import numpy as np
import kagglehub as kgh
import os

/Users/konradchrabaszcz/.pyenv/versions/3.12.10/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Setting parameters

In [2]:
RAW_DATA_PATH = (
    "/Users/konradchrabaszcz/Library/Mobile Documents/com~apple~CloudDocs/"
    "Studia/Magisterskie/II rok/III semestr/Statystyczne reguły decyzyjne/"
    "Laby/Projekt/SRD_projekt_2025/Data/Raw"
)
FINAL_DATA_PATH = (
    "/Users/konradchrabaszcz/Library/Mobile Documents/com~apple~CloudDocs/"
    "Studia/Magisterskie/II rok/III semestr/Statystyczne reguły decyzyjne/"
    "Laby/Projekt/SRD_projekt_2025/Data/Final"
)
all_dates_range = pd.date_range(
    start="2010-01-01",
    end="2013-12-31",
    freq="D"
)

## Loading data

In [3]:
test_data_df = pd.read_csv(
    filepath_or_buffer=os.path.join(RAW_DATA_PATH, "test_data.csv")
).drop(columns=["Unnamed: 0"])

In [4]:
train_data_df = pd.read_csv(
    filepath_or_buffer=os.path.join(RAW_DATA_PATH, "train_data.csv")
).drop(columns=["Unnamed: 0"])

In [5]:
features_data_df = pd.read_csv(
    filepath_or_buffer=os.path.join(RAW_DATA_PATH, "features_data.csv")
).drop(columns=["Unnamed: 0"])

In [6]:
stores_data_df = pd.read_csv(
    filepath_or_buffer=os.path.join(RAW_DATA_PATH, "stores_data.csv")
).drop(columns=["Unnamed: 0"])

In [7]:
gdp_data_df = pd.read_csv(
    filepath_or_buffer=os.path.join(RAW_DATA_PATH, "GDP.csv")
)

In [8]:
m2_money_supply_indicator_df = pd.read_csv(
    filepath_or_buffer=os.path.join(RAW_DATA_PATH, "WM2NS.csv")
)

In [9]:
retail_trade_seasonally_adjusted_df = pd.read_excel(
    io=os.path.join(RAW_DATA_PATH, "RT_seasonally_SeriesReport-202511180455-V.xlsx"),
    sheet_name="CIDR",
    skiprows=7
)

In [10]:
retail_trade_not_seasonally_adjusted_df = pd.read_excel(
    io=os.path.join(RAW_DATA_PATH, "RT_not_seasonally_SeriesReport-202511180455-V.xlsx"),
    sheet_name="CIDR",
    skiprows=7
)

In [11]:
raw_data_files_list = os.listdir(RAW_DATA_PATH)
raw_gt_data_files_list = [file for file in raw_data_files_list if "multiTimeline" in file]

for file in raw_gt_data_files_list:
    globals()[
        f"{file.replace('.csv', '').replace('multiTimeline_', '')}_gt_data_df"
    ] = pd.read_csv(
        filepath_or_buffer=os.path.join(RAW_DATA_PATH, file)
    )

In [12]:
us_holidays_data_df = pd.read_csv(
    filepath_or_buffer=os.path.join(RAW_DATA_PATH, "us_public_holidays_2010_2013.csv")
)

## Data preparing

### Functions

In [13]:
def merge_df_with_train_and_forecasts_data(df: pd.DataFrame,
                                           train_df: pd.DataFrame,
                                           forecasts_df: pd.DataFrame) -> None:
    train_df = pd.merge(
        left=train_df,
        right=df,
        how="left",
        on="Date"
    )
    forecasts_df = pd.merge(
        left=forecasts_df,
        right=df,
        how="left",
        on="Date"
    )
    return train_df, forecasts_df

###

In [14]:
for name, df in {"train_data_final_df": train_data_df,
                 "forecasts_data_final_df": test_data_df}.items():
    globals()[name] = pd.merge(
        left=df,
        right=features_data_df,
        how="left",
        on=["Store", "Date"]
    ).merge(
        right=stores_data_df,
        how="left",
        on="Store"
    ).drop(
        columns="IsHoliday_y"
    ).rename(
        columns={"IsHoliday_x": "IsHoliday"}
    )
    globals()[name]["Date"] = (
        pd.to_datetime(
            globals()[name]["Date"],
            format="%Y-%m-%d"
        )
        +
        pd.DateOffset(days=2)
    ).dt.date

In [15]:
gt_dfs = {
    gt_df_name: gt_df
    for gt_df_name, gt_df in globals().items()
    if isinstance(gt_df, pd.DataFrame) and "_gt_" in gt_df_name
}

for gt_df_name, gt_df in gt_dfs.items():
    gt_df = gt_df.reset_index()
    gt_df.rename(columns={"index": "Date"}, inplace=True)
    gt_df = gt_df.iloc[1:]
    gt_df.columns = ["Date", gt_df_name.replace("_data_df", "")]
    gt_df["Date"] = pd.to_datetime(
        arg=gt_df["Date"],
        format="%Y-%m-%d"
    ).dt.date
    train_data_final_df, forecasts_data_final_df = (
        merge_df_with_train_and_forecasts_data(df=gt_df,
                                               train_df=train_data_final_df,
                                               forecasts_df=forecasts_data_final_df)
    )

In [16]:
for name, rt_df in {"Retail_Trade_Not_Seasonally_Adjusted": retail_trade_not_seasonally_adjusted_df,
                    "Retail_Trade_Seasonally_Adjusted": retail_trade_seasonally_adjusted_df}.items():
    rt_df = rt_df.iloc[:, 0:2]
    rt_df.columns = ["Date", name]
    rt_df["Date"] = pd.to_datetime(rt_df["Date"], format="%b-%Y")
    rt_df["Date"] = rt_df["Date"].dt.strftime("%Y-%m-%d")
    rt_df["Date"] = pd.to_datetime(rt_df["Date"])
    rt_df.set_index("Date", inplace=True)
    rt_df = rt_df.reindex(all_dates_range)
    rt_df.ffill(inplace=True)
    rt_df = rt_df.resample("W").mean()
    rt_df = rt_df.reset_index()
    rt_df.columns = ["Date", name]
    rt_df["Date"] = rt_df["Date"].dt.date
    train_data_final_df, forecasts_data_final_df = (
        merge_df_with_train_and_forecasts_data(df=rt_df,
                                               train_df=train_data_final_df,
                                               forecasts_df=forecasts_data_final_df)
    )

In [17]:
m2_money_supply_indicator_df.columns = ["Date", "m2_money_supply_indicator"]
m2_money_supply_indicator_df["Date"] = (pd.to_datetime(
    m2_money_supply_indicator_df["Date"],
    format="%Y-%m-%d"
) - pd.DateOffset(days=1)).dt.date
train_data_final_df, forecasts_data_final_df = (
    merge_df_with_train_and_forecasts_data(df=m2_money_supply_indicator_df,
                                           train_df=train_data_final_df,
                                           forecasts_df=forecasts_data_final_df)
)

In [18]:
gdp_data_df.columns = ["Date", "GDP"]
gdp_data_df["Date"] = pd.to_datetime(
    arg=gdp_data_df["Date"],
    format="%Y-%m-%d"
)
gdp_final_data_df = gdp_data_df.set_index("Date")
gdp_final_data_df = gdp_final_data_df.reindex(all_dates_range)
gdp_final_data_df.ffill(inplace=True)
gdp_final_data_df = gdp_final_data_df.resample("W").mean()
gdp_final_data_df = gdp_final_data_df.reset_index()
gdp_final_data_df.rename(columns={"index": "Date"}, inplace=True)
gdp_final_data_df["Date"] = gdp_final_data_df["Date"].dt.date
train_data_final_df, forecasts_data_final_df = (
    merge_df_with_train_and_forecasts_data(df=gdp_final_data_df,
                                           train_df=train_data_final_df,
                                           forecasts_df=forecasts_data_final_df)
)

In [19]:
us_holidays_final_data_df = us_holidays_data_df.filter(items=["date", "localName"])
us_holidays_final_data_df.columns = ["Date", "HolidayName"]
us_holidays_final_data_df["Date"] = pd.to_datetime(
    arg=us_holidays_final_data_df["Date"],
    format="%Y-%m-%d"
)
us_holidays_final_data_df["Date"] = (
    us_holidays_final_data_df["Date"]
    - pd.to_timedelta((us_holidays_final_data_df["Date"].dt.weekday + 1) % 7, unit="D")
).dt.date
train_data_final_df, forecasts_data_final_df = (
    merge_df_with_train_and_forecasts_data(df=us_holidays_final_data_df,
                                           train_df=train_data_final_df,
                                           forecasts_df=forecasts_data_final_df)
)

## Saving final data

In [20]:
train_data_final_df.to_csv(
    path_or_buf=os.path.join(FINAL_DATA_PATH, "train_data_final_df.csv")
)

In [21]:
forecasts_data_final_df.to_csv(
    path_or_buf=os.path.join(FINAL_DATA_PATH, "forecasts_data_final_df.csv")
)